In [110]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import json

import io
import re

In [111]:
url = 'https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A005930'

request = requests.get(url)
soup = BeautifulSoup(request.text, 'html.parser')

In [112]:
tables = soup.find_all('table')

market_conditions = tables[0]

In [113]:
df = pd.read_html(io.StringIO(str(market_conditions)))
original_dict = df[0].dropna().to_dict(orient='tight')
my_list = original_dict['data']
values_list = []
for text in my_list:
  first_values = text[1::2]
  final_values = []
  for value in first_values:
    if '/' in value:
      split_values = value.replace(' ', '').split('/')
      final_values.extend(split_values)
    else:
      final_values.append(value)
  values_list.append(final_values)
values = [item for sublist in values_list for item in sublist]
json_dict = {
    "종가(원)": values[0],
    "전일대비(원)": values[1],
    "수익률(%)": values[2],
    "거래량(주))": values[3],
    "최고가(52주)": values[4],
    "최저가(52주)": values[5],
    "거래대금(억원)": values[6],
    "수익률(1M)": values[7],
    "수익률(3M)": values[8],
    "수익률(6M)": values[9],
    "수익률(1Y)": values[10],
    "외국인지분율(%)": values[11],
    "시가총액(상장예정포함,억원)": values[12],
    "베타(1년)": values[13],
    "시가총액(보통주,억원)": values[14],
    "액면가(원)": values[15],
    "발행주식수(보통주)": values[16],
    "발행주식수(우선주)": values[17],
    "종가(NXT)": values[18],
    "유동주식수(주)": values[19],
    "유동비율(%)": values[20],
    "거래량(NXT, 주)": values[21],
    "거래대금(NXT,억원)": values[22],
}
market_status = json.dumps(json_dict, ensure_ascii=False, indent=4)

In [114]:
df = pd.read_html(io.StringIO(str(tables[1])))[0]
earning_issue = df.to_json(orient='records', force_ascii=False, indent=4)
print(earning_issue)

[
    {
        "잠정실적발표예정일":"미정",
        "예상실적(영업이익, 억원)":88501,
        "3개월전예상실적대비(%)":-5.84,
        "전년동기대비(%)":-3.63
    }
]


In [115]:
df = pd.read_html(io.StringIO(str(tables[2])))[0]
df.set_index('운용사명', inplace=True)
df.to_csv('005930_holdings_status.csv')

In [116]:
df = pd.read_html(io.StringIO(str(tables[3])))[0]
df.dropna(inplace=True)
df.set_index('항목', inplace=True)
df.to_csv('005930_governance.csv')

In [131]:
df = pd.read_html(io.StringIO(str(tables[4])))[0]
df = df.fillna(0)
df.set_index('주주구분', inplace=True)
df.to_csv('005930_shareholders.csv')

In [121]:
# tables[5] (기업어음)은 데이터 부족으로 건너뜀
df = pd.read_html(io.StringIO(str(tables[6])))[0]
bond = df.to_json(orient='records', force_ascii=False, indent=4)
print(bond)

[
    {
        "KIS":"AAA [2004\/05\/25]",
        "KR":"AAA [2004\/06\/17]",
        "NICE":"AAA [2004\/06\/01]"
    }
]


In [124]:
df = pd.read_html(io.StringIO(str(tables[7])))[0]
analysis = df.to_json(orient='records', force_ascii=False, indent=4)
print(analysis)

[
    {
        "투자의견":4.0,
        "목표주가":84958,
        "EPS":4355,
        "PER":16.9,
        "추정기관수":24
    }
]


In [126]:
df = pd.read_html(io.StringIO(str(tables[8])))[0]
df.set_index('구분', inplace=True)
df.to_csv('005930_industry_comparison.csv')

          구분        삼성전자   코스피 전기·전자        KOSPI
0       시가총액  4345014.00  9534432.00  27489140.00
1        매출액  3008709.00  6133815.00  33576415.00
2       영업이익   327260.00   627085.00   2495513.00
3     EPS(원)     4950.00    22627.20      7000.80
4        PER       10.75       11.92        11.29
5  EV/EBITDA        4.84        5.72         6.44
6        ROE        9.03        9.81         7.48
7      배당수익률        2.72        1.75         2.28
8     베타(1년)        1.23        1.22         1.00


In [150]:
# tables[9]은 tables[8]과 동일한 테이블
df = pd.read_html(io.StringIO(str(tables[10])))[0]
df = df.fillna('없음')
df['IFRS'] = df['IFRS(연결)']
df.set_index('IFRS', inplace=True)
df_annual = df['Annual']
df_netquarter = df['Net Quarter']
print(df_annual, df_netquarter)

               2022/12     2023/12     2024/12 2025/12(E)
IFRS                                                     
매출액         3022314.00  2589355.00  3008709.00  3156607.0
영업이익         433766.00    65670.00   327260.00   288724.0
영업이익(발표기준)   433766.00    65670.00   327260.00         없음
당기순이익        556541.00   154871.00   344514.00   301945.0
지배주주순이익      547300.00   144734.00   336214.00   293663.0
비지배주주순이익       9241.00    10137.00     8300.00         없음
자산총계        4484245.00  4559060.00  5145319.00  5317587.0
부채총계         936749.00   922281.00  1123399.00  1088080.0
자본총계        3547496.00  3636779.00  4021921.00  4229506.0
지배주주지분      3451861.00  3532338.00  3916876.00  4128245.0
비지배주주지분       95635.00   104441.00   105045.00   101262.0
자본금            8975.00     8975.00     8975.00     8966.0
부채비율             26.41       25.36       27.93      25.73
유보율           38360.25    39256.91    43743.26         없음
영업이익률            14.35        2.54       10.88       9.15
지배주주순이익률      